# Description
This kernel performs inference for [PANDA training for EffNetB0 Regression](https://www.kaggle.com/fanconic/panda-training-for-effnetb0-regression).

It is originially based on @iafoss training and inference Kernel. Thank you very much for the original submission!

In [ ]:
import cv2
from tqdm import tqdm_notebook as tqdm
import fastai
from fastai.vision import *
import os
from mish_activation import *
import warnings
warnings.filterwarnings("ignore")
import skimage.io
from skimage.transform import resize
import numpy as np
import pandas as pd
import cv2

import sys
package_path = '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master'
sys.path.append(package_path)
from efficientnet_pytorch import EfficientNet

In [ ]:
os.listdir('../input/panda-training-for-effnetb0-regression')

In [ ]:
DATA = '../input/prostate-cancer-grade-assessment/test_images'
TEST = '../input/prostate-cancer-grade-assessment/test.csv'
SAMPLE = '../input/prostate-cancer-grade-assessment/sample_submission.csv'
MODELS = [f'../input/panda-training-for-effnetb0-regression/models/model.pth']

tile_sz = 128
img_sz = 224
bs = 2
N = 12
nworkers = 2

# Model

In [ ]:
class Model(nn.Module):
    def __init__(self, n=6, pre=True):
        super().__init__()
        
        # Load model backbone
        model = EfficientNet.from_name('efficientnet-b0')
        
        # Encoder, runs through the pretrained efficientnet
        self.enc = model
        
        # Neural network head. After running through the neural network, this is the transfer
        nc = list(model.children())[-1].in_features
        self.head = nn.Sequential(AdaptiveConcatPool2d(),
                                  Flatten(),
                                  nn.Linear(2*nc,512), 
                                  Mish(),
                                  nn.BatchNorm1d(512), 
                                  nn.Dropout(0.5),
                                  nn.Linear(512,1))
        
        
    def forward(self, x):
        """ Forward run through the neural network
        Params:
        x: torch tensor - input
        
        returns: output after feed forward.
        """
        # Reshape array
        shape = x.shape
        n = shape[1]
        x = x.view(-1,shape[2],shape[3],shape[4])
        
        #x: bs*N x 3 x 224 x 224
        # Go through convolutional layers
        x = self.enc.extract_features(x)
        
        shape = x.shape
        x = x.view(-1,n,shape[1],shape[2],shape[3]).permute(0,2,1,3,4).contiguous()\
          .view(-1,shape[1],shape[2]*n,shape[3])
        
        # Go through classifier
        x = self.head(x)
        return x

In [ ]:
models = []
for path in MODELS:
    state_dict = torch.load(path,map_location=torch.device('cpu'))['model']
    model = Model()
    model.load_state_dict(state_dict)
    model.float()
    model.eval()
    model.cuda()
    models.append(model)

del state_dict

# Data

In [ ]:
def tile(img, sz):
    shape = img.shape
    
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],constant_values=255)
    
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
        
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    
    return img


mean = torch.tensor([1.0-0.90949707, 1.0-0.8188697, 1.0-0.87795304])
std = torch.tensor([0.36357649, 0.49984502, 0.40477625])


class PandaDataset(Dataset):
    def __init__(self, path, test):
        self.path = path
        self.names = list(pd.read_csv(test).image_id)

        
    def __len__(self):
        return len(self.names)

    
    def __getitem__(self, idx):
        name = self.names[idx]
        img = skimage.io.MultiImage(os.path.join(DATA,name+'.tiff'))[-1]
        tiles = tile(img, tile_sz)
        tiles = torch.Tensor(tiles)
        tiles = tiles.permute(0,3,1,2)
        tiles = nn.functional.interpolate(tiles, size = img_sz, mode = 'bilinear')
        tiles = 1- tiles/255.0
        tiles = (tiles - mean[...,None,None])/std[...,None,None]
        return tiles, name

In [ ]:
"""# Only for testing with train folder
ds = PandaDataset(DATA,TEST)
dl = DataLoader(ds, batch_size=bs, num_workers=nworkers, shuffle=False)

for x,y in dl:
    fig, axs = plt.subplots(2,12, figsize=(25, 6), facecolor='w', edgecolor='k')
    axs = axs.ravel()
    for j,batch in enumerate(x):
        for i,t in enumerate(batch):
            axs[j*12+i].imshow(t.permute(1,2,0).numpy())
        if j == 1:
            break
    break"""


# Prediction

In [ ]:
class KappaOptimizer(nn.Module):
    def __init__(self):
        super().__init__()
        self.coef = [0.5, 1.5, 2.5, 3.5]
        # define score function:
        self.func = self.quad_kappa
    
    
    def predict(self, preds):
        return self._predict(self.coef, preds)

    
    @classmethod
    def _predict(cls, coef, preds):
        if type(preds).__name__ == 'Tensor':
            y_hat = preds.clone().view(-1)
        else:
            y_hat = torch.FloatTensor(preds).view(-1)

        for i,pred in enumerate(y_hat):
            if   pred < coef[0]: y_hat[i] = 0
            elif pred < coef[1]: y_hat[i] = 1
            elif pred < coef[2]: y_hat[i] = 2
            elif pred < coef[3]: y_hat[i] = 3
            else:                y_hat[i] = 4
        return y_hat.int()
    
    
    def quad_kappa(self, preds, y):
        return self._quad_kappa(self.coef, preds, y)

    
    @classmethod
    def _quad_kappa(cls, coef, preds, y):
        y_hat = cls._predict(coef, preds)
        
        try:
            return cohen_kappa_score(y, y_hat, weights='quadratic')
        except:
            return cohen_kappa_score(y.cpu(), y_hat.cpu(), weights='quadratic')

    
    def fit(self, preds, y):
        ''' maximize quad_kappa '''
        neg_kappa = lambda coef: -self._quad_kappa(coef, preds, y)
        opt_res = sp.optimize.minimize(neg_kappa, x0=self.coef, method='nelder-mead',
                                       options={'maxiter':100, 'fatol':1e-20, 'xatol':1e-20})
        self.coef = opt_res.x

        
    def forward(self, preds, y):
        ''' the pytorch loss function '''
        return torch.tensor(self.quad_kappa(preds, y))


kappa_opt = KappaOptimizer()

# Optimized Thresholds from previous training
kappa_opt.coef = [0.55589, 1.635796, 2.427105, 3.058962]

In [ ]:
sub_df = pd.read_csv(SAMPLE)
if os.path.exists(DATA):
    ds = PandaDataset(DATA,TEST)
    dl = DataLoader(ds, batch_size=bs, num_workers=nworkers, shuffle=False)
    names,preds = [],[]
    
    with torch.no_grad():
        for x,y in tqdm(dl):
            x = x.cuda()
            #trafos = [x]
            
            #dihedral TTA
            trafos = [x,
                      x.flip(-1),
                      x.flip(-2),
                      x.flip(-1,-2),
                      x.transpose(-1,-2),
                      x.transpose(-1,-2).flip(-1),
                      x.transpose(-1,-2).flip(-2),
                      x.transpose(-1,-2).flip(-1,-2)]
            
            
            # Reshape input
            x = torch.stack(trafos,1)
            x = x.view(-1,N,3,img_sz, img_sz)
            
            # Make predictions
            p = [model(x) for model in models]
            p = torch.stack(p,1)
            
            # Take mean/median of predictions
            p = p.view(bs,len(trafos)*len(models),-1).median(1).values
            p = kappa_opt.predict(p).cpu()
            #p = p.view(bs,len(trafos)*len(models),-1).mean(1).argmax(-1).cpu()
            
            names.append(y)
            preds.append(p)
    
    names = np.concatenate(names)
    preds = torch.cat(preds).numpy()
    sub_df = pd.DataFrame({'image_id': names, 'isup_grade': preds})
    sub_df.to_csv('submission.csv', index=False)
    sub_df.head()

In [ ]:
"""# Only for testing with train folder
from sklearn.metrics import cohen_kappa_score, confusion_matrix

sub_df = pd.DataFrame({'image_id': names, 'isup_grade': preds})
df = pd.read_csv(TEST)
df.drop(["isup_grade"], axis =1)
df = df.merge(sub_df, left_on="image_id", right_on="image_id")
df = df.dropna()

t = df.isup_grade_x
p = df.isup_grade_y
print(cohen_kappa_score(t,p,weights='quadratic'))
print(confusion_matrix(t,p))"""

In [ ]:
sub_df.to_csv("submission.csv", index=False)
sub_df.head()